# Multilingual scoring

Score verified Reddit posts using multilingual sentiment (XLM-RoBERTa) and a simple engagement metric.

In [5]:
import numpy as np
import pandas as pd
import duckdb
from pathlib import Path
from transformers import pipeline


## Load data from SQL

Query verified submissions plus their comments directly into pandas using DuckDB.

Loaded 3702 posts


In [11]:
posts_df = pd.read_csv("/Users/ivantyshchenko/Projects/Python/DataPatron/data/reddit.csv")
posts_df

,post_id,post_author,post_created_at,subreddit,title,selftext,post_score,upvote_ratio,num_comments,url,domain,comments_content,comments_scores
0,1lnl1el,humanlikecorvus,2025-06-29 20:57:14+03:00,UkrainianConflict,JOINT SUBREDDIT FUNDRAISER FOR UKRAINE X UNITED24,"Hello users of r/UkrainianConflict ,\n\nFor th...",104,0.98,6,https://i.redd.it/ik3j60vfmw9f1.png,i.redd.it,['[removed]'\n 'Hi! United24s initial goal of ...,[1 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1,1met4oq,Distinct-Quit-3223,2025-08-01 13:42:32+03:00,CombatFootage,FPV's of the 14th Operational Assignment Briga...,NaN,349,0.96,9,https://v.redd.it/om1h95v40egf1,/r/CombatFootage/comments/1met4oq/fpvs_of_the_...,['Please keep the [community guidelines](https...,[ 1 1 1 11 29 9 10 1 1 1 1 1 11 29 9 ...
2,1metez4,Mil_in_ua,2025-08-01 13:59:17+03:00,UkrainianConflict,Germany to Supply Two Patriot Systems to Ukrai...,NaN,90,0.98,1,https://militarnyi.com/en/news/germany-to-supp...,militarnyi.com,"[""Please take the time to read [the rules](/r/...",[1 1]
3,1mevbmj,UNITED24Media,2025-08-01 15:35:50+03:00,UkrainianConflict,"Ukraine Pushes East in Donetsk Region, Forcing...",NaN,145,0.99,3,https://united24media.com/latest-news/ukraine-...,united24media.com,"[""Please take the time to read [the rules](/r/...",[1 2 3 1 2 3]
4,1mevf5c,Pristine_Squirrel_27,2025-08-01 15:40:20+03:00,UkrainianConflict,Germany to deliver two Patriot systems to Ukra...,NaN,206,1.00,2,https://www.reuters.com/world/europe/germany-d...,reuters.com,"[""Please take the time to read [the rules](/r/...",[ 1 10 1 10]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697,1lfftln,Econ_Orc,2025-06-19 19:55:02+03:00,europe,Denmark to push for Ukraines EU membership dur...,NaN,1221,0.97,61,https://www.reuters.com/sustainability/climate...,reuters.com,NaN,NaN
3698,1lfqv6t,SpecialK_Anon,2025-06-20 03:45:54+03:00,ukraine,I wrote this letter to my representatives in c...,**Find your representatives here:** [**https:/...,96,1.00,1,https://www.reddit.com/r/ukraine/comments/1lfq...,self.ukraine,NaN,NaN
3699,1lgwjya,HellYeahDamnWrite,2025-06-21 16:27:17+03:00,UkrainianConflict,Putin declares ‘all of Ukraine is ours’ in lat...,NaN,253,0.95,41,https://www.independent.co.uk/news/world/europ...,independent.co.uk,NaN,NaN
3700,1lj5jqe,BezugssystemCH1903,2025-06-24 11:20:01+03:00,de,Sichere ukrainische Gebiete - Schweiz schränkt...,NaN,27,0.89,0,https://www.srf.ch/news/schweiz/sichere-ukrain...,srf.ch,NaN,NaN


## Set up multilingual sentiment model

XLM-RoBERTa sentiment model with full-score output for downstream scoring.

In [13]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

def preprocess(text):
  parts = []
  for t in text.split(" "):
      if t.startswith("@") and len(t) > 1:
          t = "@user"
      if t.startswith("http"):
          t = "http"
      parts.append(t)
  return " ".join(parts)

model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

text = preprocess("Good night 😊")
encoded = tokenizer(text, return_tensors="pt")
scores = model(**encoded).logits[0].detach().numpy()
scores = softmax(scores)

for idx in scores.argsort()[::-1]:
  print(f"{config.id2label[idx]} {scores[idx]:.4f}")

AttributeError: 'NoneType' object has no attribute 'endswith'

In [ ]:
MODEL_NAME = 'cardiffnlp/twitter-xlm-roberta-base-sentiment'

sentiment_pipeline = pipeline(
    'sentiment-analysis',
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    top_k=None,
    truncation=True,
    max_length=512
)

TOXIC_KEYWORDS = {
    'fake', 'bot', 'propaganda', 'liar', 'stupid', 'idiot', 'scam',
    'фейк', 'бот', 'брехня', 'пропаганда', 'іпсо', 'дурень', 'ідіот',
    'вранье', 'ложь'
}


## Metric helpers

Plain functions for multilingual sentiment scoring, efficiency, and UCQS.

In [ ]:
def to_multilingual_score(text: str) -> float:
    if not isinstance(text, str) or not text.strip():
        return 0.0
    try:
        results = sentiment_pipeline(text)[0]
        scores = {item['label'].lower(): item['score'] for item in results}
        return scores.get('positive', 0.0) - scores.get('negative', 0.0)
    except Exception as exc:
        print(f'Sentiment fallback for text due to error: {exc}')
        return 0.0

def calculate_efficiency(post_score, num_comments, upvote_ratio) -> float:
    score = post_score if pd.notnull(post_score) else 0
    comments = num_comments if pd.notnull(num_comments) else 0
    ratio = upvote_ratio if pd.notnull(upvote_ratio) else 0.5
    raw_engagement = score + (comments * 2.0)
    log_engagement = np.log1p(raw_engagement)
    return round(log_engagement * ratio, 2)

def calculate_ucqs(comments_content) -> float:
    if not isinstance(comments_content, list) or not comments_content:
        return 50.0
    sentiments = []
    toxic_count = 0
    valid_comments = 0
    for text in comments_content:
        if not isinstance(text, str) or not text.strip():
            continue
        valid_comments += 1
        score = to_multilingual_score(text[:512])
        sentiments.append(score)
        lower = text.lower()
        if any(k in lower for k in TOXIC_KEYWORDS) or score < -0.7:
            toxic_count += 1
    if not sentiments:
        return 50.0
    s_avg = float(np.mean(sentiments))
    s_var = float(np.std(sentiments))
    t_ratio = toxic_count / valid_comments if valid_comments else 0
    ucqs = 50 + (s_avg * 40) - (s_var * 20) - (t_ratio * 30)
    return float(np.clip(ucqs, 0, 100))


## Apply metrics

Compute both scores and inspect the result set.

In [ ]:
posts_df['efficiency'] = posts_df.apply(
    lambda row: calculate_efficiency(row['post_score'], row['num_comments'], row['upvote_ratio']),
    axis=1
)

posts_df['ucqs'] = posts_df['comments_content'].apply(calculate_ucqs)

posts_df[['post_id', 'efficiency', 'ucqs']].head()


,post_id,post_author,post_created_at,subreddit,title,selftext,post_score,upvote_ratio,num_comments,url,domain,comments_content,comments_scores
0,1lnl1el,humanlikecorvus,2025-06-29 20:57:14+03:00,UkrainianConflict,JOINT SUBREDDIT FUNDRAISER FOR UKRAINE X UNITED24,"Hello users of r/UkrainianConflict ,\n\nFor th...",104,0.98,6,https://i.redd.it/ik3j60vfmw9f1.png,i.redd.it,['[removed]'\n 'Hi! United24s initial goal of ...,[1 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1,1met4oq,Distinct-Quit-3223,2025-08-01 13:42:32+03:00,CombatFootage,FPV's of the 14th Operational Assignment Briga...,NaN,349,0.96,9,https://v.redd.it/om1h95v40egf1,/r/CombatFootage/comments/1met4oq/fpvs_of_the_...,['Please keep the [community guidelines](https...,[ 1 1 1 11 29 9 10 1 1 1 1 1 11 29 9 ...
2,1metez4,Mil_in_ua,2025-08-01 13:59:17+03:00,UkrainianConflict,Germany to Supply Two Patriot Systems to Ukrai...,NaN,90,0.98,1,https://militarnyi.com/en/news/germany-to-supp...,militarnyi.com,"[""Please take the time to read [the rules](/r/...",[1 1]
3,1mevbmj,UNITED24Media,2025-08-01 15:35:50+03:00,UkrainianConflict,"Ukraine Pushes East in Donetsk Region, Forcing...",NaN,145,0.99,3,https://united24media.com/latest-news/ukraine-...,united24media.com,"[""Please take the time to read [the rules](/r/...",[1 2 3 1 2 3]
4,1mevf5c,Pristine_Squirrel_27,2025-08-01 15:40:20+03:00,UkrainianConflict,Germany to deliver two Patriot systems to Ukra...,NaN,206,1.00,2,https://www.reuters.com/world/europe/germany-d...,reuters.com,"[""Please take the time to read [the rules](/r/...",[ 1 10 1 10]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697,1lfftln,Econ_Orc,2025-06-19 19:55:02+03:00,europe,Denmark to push for Ukraines EU membership dur...,NaN,1221,0.97,61,https://www.reuters.com/sustainability/climate...,reuters.com,NaN,NaN
3698,1lfqv6t,SpecialK_Anon,2025-06-20 03:45:54+03:00,ukraine,I wrote this letter to my representatives in c...,**Find your representatives here:** [**https:/...,96,1.00,1,https://www.reddit.com/r/ukraine/comments/1lfq...,self.ukraine,NaN,NaN
3699,1lgwjya,HellYeahDamnWrite,2025-06-21 16:27:17+03:00,UkrainianConflict,Putin declares ‘all of Ukraine is ours’ in lat...,NaN,253,0.95,41,https://www.independent.co.uk/news/world/europ...,independent.co.uk,NaN,NaN
3700,1lj5jqe,BezugssystemCH1903,2025-06-24 11:20:01+03:00,de,Sichere ukrainische Gebiete - Schweiz schränkt...,NaN,27,0.89,0,https://www.srf.ch/news/schweiz/sichere-ukrain...,srf.ch,NaN,NaN
